In [10]:
import joblib
from sklearn.cluster import KMeans
from scipy.spatial import distance
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.cluster.hierarchy import ward, dendrogram, fcluster, single, complete
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score

# feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer 

# data
import pandas as pd
import numpy as np
import os
import json

# viz
import matplotlib.pyplot as plt
%matplotlib inline

# other
from tqdm import tqdm
from copy import deepcopy
import time


nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import re
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    print(tokens)
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # print(tokens)
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    # print(filtered_tokens)
    return filtered_tokens

In [2]:
def load_models(Path_Kmeans, Path_tfidf):
  km = joblib.load(Path_Kmeans)
  tfidf_vec = joblib.load(Path_tfidf)
  return km,tfidf_vec


In [18]:
def predict_cluster(km_model, tfidf_model, data):
  tfidf_matrix = tfidf_model.transform(data)
  lables= km_model.predict(tfidf_matrix)
  return lables[0]



In [21]:
def find_nearset_cluster(km_model, tfidf_model,data_clusters):
  if len(np.unique(data_clusters))==1:
    return np.unique(data_clusters)
  else:
    centers=  km_model.cluster_centers_ 
    centroids_midpoint=np.zeros((len(centers[0])))
    
    for val in np.unique(data_clusters):
      for i in range (len(centers[0])):
        centroids_midpoint[i] += centers[val][i]
    centroids_midpoint=centroids_midpoint/len(np.unique(data_clusters))
    distances=[]
    for i in range(len(centers)):
      distances.append(distance.euclidean(centroids_midpoint, centers[i]))
    return np.argmin(np.array(distances))





In [12]:
km_model,tfidf_model=load_models('/content/drive/MyDrive/HackZurich_2022/Saved Models/Km_model.pkl',  '/content/drive/MyDrive/HackZurich_2022/Saved Models/tfidf_vector_model.pkl')

In [13]:
data_point={'name': 'Michael Delker',
 'interests': 'movie,adventure,science,politics,fashion,culture',
 'journey_frequency': 702,
 'cluster': None}

In [19]:
print('predict cluster')
print(predict_cluster(km_model,tfidf_model,[data_point['interests']]))

predict cluster
96


In [22]:
Cluster_list=[96,1,99,98,100,30,51,41]
print('Midpoint cluster is')
print(find_nearset_cluster(km_model,tfidf_model,Cluster_list))

Midpoint cluster is
320
